## Computer Architecture Essentials

+ Let's review some aspects of computer hardware & architecture to understand the preceding timings.

---

### Hardware considerations

<center>
    <img src='./img/hardware-0.svg' width=50%>
</center>

---

+ The *CPU* or *Central Processing Unit* is where operations on data are actually executed.
+ It's convenient to imagine a CPU as a clerical worker at a desk, manipulating paperwork.

<center>
    <img src='./img/hardware-1.svg' width=50%>
</center>

+ There's actually another layer to it.
+ Modern CPUs have local very fast *cache* memory.
+ With the clerical worker metaphor in mind, the cache would be like the desk surface & drawers that contain papers close by, ready-to-access.

---

<center>
    <img src='./img/hardware-2.svg' width=50%>
</center>

+ The next level of data storage is *Random Access Memory* or (*RAM*).
+ RAM is volatile in that data vanishes when the machine shuts down.
+ A computer's main memory can be thought of as a bookcase near the clerk.
+ It has more storage and takes more time to find papers.

---

<center>
    <img src='./img/hardware-3.svg' width=50%>
</center>

+ Disk drives are *permanent* storage
+ Rotational hard disk drives are still in use, but Solid state drives are much faster.
+ We can view disk storage as a filing cabinet with more space for our clerk's paperwork.

---

<center>
    <img src='./img/hardware-4.svg' width=50%>
</center>

+ Finally, when data isn't stored on disk, we can always go to the internet.
+ Network storage is much slower depending on how far data has to travel.
+ Rounding out our clerical worker metaphor, the internet is like a library or an archive that the clerk visits to find archived files.

---

### NumPy & Pandas data structures

<center>
<img src='./img/pydata_ecosystem.png' width=60%></img>
</center>

+ With a model of how computers store & work with data in mind, let's review some things about NumPy & Pandas.
+ Both packages are at the base of the Python scientific software ecosystem.

---

#### What problem does NumPy solve?

+ Python lists:
  + more overhead for elementary tasks
  + e.g., extract 7th book from right

<center>
<img src=https://1.bp.blogspot.com/-5hiJFTt7_bk/XhWrJXoL_NI/AAAAAAAB4ys/Kw50Yf81B0IOIzOPpUM1-rGFcN021IytgCLcBGAsYHQ/s1600/Bookshelf%2B-%2BFiction%2B1.jpg width=40%></img>
</center>

+ Superficially, it seems that Python lists are adequate ordered containers for data.
+ They are heterogeneous, i.e., can contain elements of distinct type or size.
+ So list operations like indexing, search, and so on are similar to searching on this shelf of books of different shapes.
+ As an example, consider finding the 7th book from the right in this image of nonuniformly shaped books.

---

+ NumPy arrays:
  + reduces overhead for elementary tasks
  + e.g., extract 7th book from right

<center>
<img src=https://i1.wp.com/farm2.staticflickr.com/1769/43025256801_789480a50a_b.jpg width=40%></img>
</center>

+ NumPy arrays, by contrast, are *homogenous*
+ They contain elements all of the same datatype & size in memory.
+ So array operations are more like working with a shelf of encyclopedias of identical width.
+ Finding the seventh book from the right is noticeably easier in this instance.

---

#### Performance comparisons

In [1]:
import pandas as pd, numpy as np
import pickle as pkl
pd.set_option('display.precision', 2, 'display.min_rows', 6)

In [2]:
from s3fs import S3FileSystem
s3 = S3FileSystem(anon=True)

In [3]:
DATA_ROOT = 'bodo-examples-data/bodo-training-fundamentals/DATA'
for FILE in ['ARRAY/data.npy', 'ARRAY/data.pkl']:
    print(f'{FILE}:\t{s3.du(f"{DATA_ROOT}/{FILE}")/(1024**2):.2f} MiB')
    s3.get(rpath=f'{DATA_ROOT}/{FILE}', lpath=f'./{FILE}')

ARRAY/data.npy:	381.47 MiB
ARRAY/data.pkl:	737.30 MiB


---

+ To see this concretely, let's import some standard packages...

---

+ ... let's create an object `s3` so we can examine the S3 bucket files...

---

+ ...and let's look at the sizes of two particular files:
 + `data.npy` contains a compressed NumPy array;
 + `data.pkl` contains a pickled Python list.
+ Notice this loop also downloads the files locally; this can take a few minutes
  + ...about 6 minutes on my network connection.

In [4]:
data = np.load('./ARRAY/data.npy')

In [5]:
print(f'{len(data):,d} elements\n{data.nbytes/(1024**2):.2f} MiB memory\ndtype {data.dtype}')
print(data)

200,000,000 elements
381.47 MiB memory
dtype int16
[-862  800 -919 ...   31  565 -402]


---

+ The file `data.npy` contains 200 million 16-bit integers.
---
+ When loaded into memory, they occupy about 380 MiB.

In [6]:
%time print(f'Max:\t{data.max():6d}')

Max:	  1000
CPU times: user 133 ms, sys: 0 ns, total: 133 ms
Wall time: 132 ms


In [7]:
%time print(f'Min:\t{data.min():6d}')

Min:	 -1000
CPU times: user 126 ms, sys: 0 ns, total: 126 ms
Wall time: 125 ms


In [8]:
%time print(f'Mean:\t{data.mean():6.2f}')

Mean:	 -0.88
CPU times: user 110 ms, sys: 5 µs, total: 110 ms
Wall time: 108 ms


---

+ As mentioned, NumPy is set up to deal with homogeneous arrays of data efficiently.
---
+ Elementary array operations like computing a maximum or a minimum...
---
+ ... or an average can be done in a hundred milliseconds or so.

In [9]:
with open('./ARRAY/data.pkl', 'rb') as in_file:
    data_list = pkl.load(in_file)
print(f'{len(data_list):,d} elements\nType: {type(data_list)}')

print(f'{data_list[:3]} ... {data_list[-3:]}')

200,000,000 elements
Type: <class 'list'>
[-862, 800, -919] ... [31, 565, -402]


+ By contrast, the pickled file `data.pkl` contains the same data stored as a list or Python integers.
+ The size on disk of this file is about double that of the NumPy array.

---

In [10]:
%time print(f'Max:\t{max(data_list):6d}')

Max:	  1000
CPU times: user 3.56 s, sys: 7.87 ms, total: 3.57 s
Wall time: 3.54 s


In [11]:
%time print(f'Min:\t{min(data_list):6d}')

Min:	 -1000
CPU times: user 3.55 s, sys: 10.6 ms, total: 3.56 s
Wall time: 3.54 s


In [12]:
%time print(f'Mean:\t{sum(data_list)/len(data_list):6.2f}')

Mean:	 -0.88
CPU times: user 2.47 s, sys: 6.74 ms, total: 2.48 s
Wall time: 2.46 s


---

+ Computation times for the maximum...
---
+ ... the minimum...
---
+ ... and the average of the list of integers is much longer (a few seconds each).
+ Remember, Python lists are generically heterogeneous, so computation done on Python objects is generically slower.

+ Pandas Series: array with labelled index
<center>
    <tr>
    <td><img align='center', src='./img/weather_series.png', width="50%"></img></td>
    </tr>
</center>

+ These same considerations were extended from NumPy in developing Pandas.
+ Pandas Series are like one-dimensional homogeneous NumPy arrays with a labelled Index.
  + A common use case is for *Time Series* like this series of temperature values indexed by time.
  + Pandas Indexes are optimized for high-performance allowing labelled access to homogeneous one-dimensional arrays.

---

+ Pandas DataFrame: distinct Series, common index

<center>
<img src='./img/weather_frame.png'></img>
<img src='./img/stock_frame.png'></img>
</center>

+ Pandas DataFrames consist of numerous series with a common index.
  + The first DataFrame has Temperature & density values indexed by time.
  + The second DataFrame has stock ticker information: open, high, low, closing prices, & volumes traded indexed by day.
+ Notice that the columns of a DataFrame are Series.
+ DataFrames are well-suited for column-oriented computations/storage with homogenous data in columns.
+ They can have distinct data types.
  + for instance, the stock ticker DataFrame has columns of floating-point prices except for the Volume column that contains integers.

---

#### JIT compilers: interpreted vs. compiled programs

+ Intense computations in *loops over arrays*
+ NumPy/Pandas: C layer for low-level array computing
+ Bodo (& [Numba](http://numba.pydata.org/numba-doc/latest/)): *JIT (just in-time) compilation*
  + [LLVM Compiler Infrastructure](http://llvm.org/) underneath
  + Simple way to compile Python code

+ Computationally intensive operations usually involve *arrays* and *loops*
  + typically having the same operations applied to many elements.
+ As seen, explicit loops in Python are slow; NumPy & Pandas avoid this by pushing costly operations to a layer of C code.
+ Another approach is to *compile* functions into machine code earlier.
+ This is how Numba & Bodo—using the mature LLVM infrastructure—accelerates programs seen so far.

---

### Memory hierarchy

<center>
    <img src='./img/memory-hierarchy-0.svg' width=60%>
</center>

+ There's another last piece to understand the performance differences seen so far: the hierarchy of data access.

---

### Memory hierarchy

<center>
    <img src='./img/memory-hierarchy-1.svg' width=60%>
</center>

+ A typical modern CPU has a clock cycle of about 0.4 nanoseconds.
+ The precise number is not important; just on the order of a billion cycles per second.

---

### Memory hierarchy

<center>
    <img src='./img/memory-hierarchy-2.svg' width=60%>
</center>

+ Fetching data from the CPU cache takes about 0.9 to 28 nanoseconds.
+ Returning to our metaphor of the CPU as a clerical worker, fetching from cache is like getting a paper from a drawer or the desktop.

---

### Memory hierarchy

<center>
    <img src='./img/memory-hierarchy-3.svg' width=60%>
</center>

+ Retrieving data from RAM takes roughly 100 to 350 nanoseconds.
+ For our clerk, this is like looking for a book on their bookcase.

---

### Memory hierarchy

<center>
    <img src='./img/memory-hierarchy-4.svg' width=60%>
</center>

+ Looking for data on a local disk takes anywhere from 10 microseconds to 10 milliseconds.
+ That's like looking for a physical file in a filing cabinet for our clerk.

---

### Memory hierarchy

<center>
    <img src='./img/memory-hierarchy-5.svg' width=60%>
</center>

+ Finally, fetching data over the internet takes much longer—on the order of 65 milliseconds for a packet to travel round- trip between San Francisco to New York.
   + (of course, it depends on network speed and is even longer for greater geographic distances).
+ This would be like our clerk going our to the local library or an archive to find a document.

---

### Computer latency at human scale

<center>
    <img src='./img/computer-latency-0.svg' width=70%>
</center>

+ Brendan Gregg summarised this by a metaphor: *computer latency at human scale*
+ Let's pretend the our clerical worker can complete a form in one second (corresponding to a clock cycle).

---

### Computer latency at human scale

<center>
    <img src='./img/computer-latency-1.svg' width=70%>
</center>

+ In rescaled units, that means fetching from Level 1, Level 2, or Level 3 cache takes 2 seconds, 7 seconds, or a minute for the clerk.
+ Again, the precise hardware numbers don't matter as much as their relative scale.

---

### Computer latency at human scale

<center>
    <img src='./img/computer-latency-2.svg' width=70%>
</center>

+ Then fetching from memory would take between 4 and 15 minutes.

---

### Computer latency at human scale

<center>
    <img src='./img/computer-latency-3.svg' width=70%>
</center>

+ Getting data from a fast Solid State Drive would take the clerk 7 hours to 4 days...
+ ... and a traditional rotational hard disk drive access is between 1 and 9 months!

---

### Computer latency at human scale

<center>
    <img src='./img/computer-latency-4.svg' width=70%>
</center>

+ Finally, at this scale, an internet round-trip takes 5 years!

---

### Computer latency at human scale

<center>
    <img src='./img/computer-latency-5.svg' width=70%>
</center>

+ This rescaling reveals the limitations of our metaphor of a clerk in an office.
+ But it is useful to have in mind when asking why programs are slow.
+ The critical performance bottleneck is often in how data moves between devices.

---

### Summary

+ Hardware considerations
+ NumPy & Pandas data structures
+ Memory Hierarchy
+ Computer Latency at Human Scale

+ Putting it together, we can understand some of the performance differances observed so far.
+ First, artful use of NumPy & Pandas can speed up computations a lot.
+ Bodo can improves things further by compiling programs to carefully optimize motion of data between cache & RAM.
+ In looking to construct such speed-ups, it's useful to remember the hierarchy of memory & to consider computer latencies at human scales.